# **Important Installations & Dependencies**


In [ ]:
import re
import string
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
!pip install farasapy
from farasa.stemmer import FarasaStemmer
from sklearn.model_selection  import train_test_split
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score,recall_score
from sklearn.metrics import classification_report as creport
from multiprocessing import Pool

### **Load the Binary Datset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import io
df= pd.read_excel(io.BytesIO(uploaded['BinaryPublicFigures.xlsx'])) 
df= df[['Text','Label']]
df

### **Load the Multi Datset**

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Art= pd.read_excel(io.BytesIO(uploaded['Art.xlsx']))
# Art= Art[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Business= pd.read_excel(io.BytesIO(uploaded['Business.xlsx']))
# Business= Business[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Politics= pd.read_excel(io.BytesIO(uploaded['Politics.xlsx']))
# Politics= Politics[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Judiciary= pd.read_excel(io.BytesIO(uploaded['Judiciary.xlsx']))
# Judiciary= Judiciary[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Sports= pd.read_excel(io.BytesIO(uploaded['Sports.xlsx']))
# Sports= Sports[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Literature= pd.read_excel(io.BytesIO(uploaded['Literature.xlsx']))
# Literature= Literature[['Text','Label']]

In [ ]:
# Business = Business[Business['Label'] == 0]
# Literature = Literature[Writer['Label'] == 0]
# Sports = Sports[Sports['Label'] == 0]
# Judiciary = Judiciary[Judiciary['Label'] == 0]
# Politics = Politics[Politics['Label'] == 0]
# Art = Art[Art['Label'] == 0]

In [ ]:
# Literature['Label'] = Literature['Label'].replace([0],[1])
# Sports['Label'] = Sports['Label'].replace([0],[2])
# Judiciary['Label'] = Judiciary['Label'].replace([0],[3])
# Politics['Label'] = Politics['Label'].replace([0],[4])
# Art['Label'] = Art['Label'].replace([0],[5])

In [ ]:
# frames= [Literature,Sports, Judiciary, Politics, Art, Business]
# df= pd.concat(frames)

In [ ]:
df['Label'].value_counts()

In [ ]:
# df["Label"] = df["Label"].astype(str).astype(int)
# dataTypeObj = df.dtypes['Label']
# print(dataTypeObj)

# **Data Preprocessing**


In [ ]:
def clean(df):
    df = remove_diacritics(df)
    df = normalize_arabic(df)
    df = remove_punctuations(df)
    df = remove_repeating_char(df)
    df= remove_english_word_and_numbers(df)
    df=clean_space(df)
    return df

In [ ]:
arabic_punctuations = '''`÷« »×؛<>٩٨'٧٦٥٤٣٢١٠_()↗*•&^%][ـ،/:"؟.,'{}⋮≈~¦+|٪!”…“–ـ/[]%=#*+\\•~@£·_{}©^®`→°€™›♥←×§″′Â█à…“★”–●â►−¢¬░¶↑±▾	═¦║―¥▓—‹─▒：⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡₹´'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


In [ ]:
def remove_diacritics(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_diacritics(x))
    return df
def _remove_diacritics(x):
    x = str(x)
    x = re.sub(arabic_diacritics, '', x)
    return x

def normalize_arabic(df):
    df['Text'] = df['Text'].apply(lambda x: _normalize_arabic(x))
    return df
def _normalize_arabic(x):
    x = str(x)
    # added space around puncts after replace
    x = re.sub("[إأآا]", "ا", x)
    x = re.sub("ى", "ي", x)
    x = re.sub("ؤ", "ء", x)
    x = re.sub("ئ", "ء", x)
    x = re.sub("ة", "ه", x)
    x = re.sub("گ", "ك", x)
    return x

def remove_punctuations(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_punctuations(x))
    return df
def _remove_punctuations(x):
    x = str(x)
    #translator = str.maketrans(' ', ' ', punctuations_list)
    translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
    return x.translate(translator)

def remove_repeating_char(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_repeating_char(x))
    return df
def _remove_repeating_char(x):
    x = str(x)
    return re.sub(r'(.)\1+', r'\1', x)

def remove_english_word_and_numbers(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_english_word_and_numbers(x))
    return df
def _remove_english_word_and_numbers(x):
    x = str(x)
    return re.sub(r'[a-zA-Z0-9]+', '', x)

def clean_space(df):
    compiled_re = re.compile(r"\s+")
    df['Text'] = df["Text"].apply(lambda x: _clean_space(x, compiled_re))
    return df
def _clean_space(x, compiled_re):
    return compiled_re.sub(" ", x)


In [ ]:
text_a=df['Text'].values.tolist()
print(len(text_a))

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words= stopwords.words('arabic')
for index,text_ in enumerate(text_a):
    text_=" ".join(i for i in text_.split() if i not in stop_words) #Removing stopword
    text_a[index]=text_

In [ ]:
#Stemming the word
stemmer = FarasaStemmer(interactive=True)
for index,text_ in enumerate(text_a):
    text_=" ".join(stemmer.stem(i) for i in text_.split())   
    text_a[index]=text_

In [ ]:
df['Text']=text_a[:len(df)]
df.head()

In [ ]:
num_cores = 2
def df_parallelize_run(df, func, num_cores=2):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
df = df_parallelize_run(df, clean)

In [ ]:
df= shuffle(df,random_state=0)
df.head()

In [ ]:
data_train, data_test = train_test_split(df,test_size = 0.2)

In [ ]:
data_train.to_csv("data_train.csv", index=False)
!cp -R data_train.csv "/content/drive/MyDrive/DatasetCleaned"
data_test.to_csv("data_test.csv", index=False)
!cp -R data_test.csv "/content/drive/MyDrive/DatasetCleaned" 

# **Training & Testing**


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/DatasetCleaned/data_train.csv')

In [ ]:
x=dataset.iloc[:,0]
y=dataset.iloc[:,1]
X=x.to_dict()

X=[]
for d in range(len(x)):
    b=x[d]
    X.append(b) 

In [ ]:
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(X)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf= X_train_tfidf.toarray()

In [ ]:
clf= naive_bayes.BernoulliNB()
clf.fit(X_train_tfidf, y)
clf.score(X_train_tfidf, y)

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/DatasetCleaned/data_test.csv')

In [ ]:
x_test=test_data.iloc[:,0]
y_test=test_data.iloc[:,1]
X=x_test.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)  

In [ ]:
X_test_tfidf=count_vect.transform(x_test)
y_pred=clf.predict(X_test_tfidf)

In [ ]:
print("NB weighted-averaged precision-score -> ",precision_score(y_test, y_pred,average="binary", pos_label="1"))
print("NB weighted-averaged recall-score -> ",recall_score(y_test, y_pred,average="binary", pos_label="1"))
print("NB weighted-averaged F1-score -> ",f1_score(y_test, y_pred,average="binary", pos_label="1"))

In [ ]:
print(creport(y_test, y_pred,target_names=['Offensive', 'Benign'],digits=4)) # Forthe Binary dataset 
# print(creport(y_test, y_pred,target_names=['Literature,Sports, Judiciary, Politics, Art, Business'],digits=4)) // For Multi dataset